In [21]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import mlflow
from mlflow import MlflowClient
# Définir le nom de l'expérience MLflow

In [3]:
df = pd.read_csv('dpe-v2-logements-neufs.csv')

C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\4243231139.py:1: DtypeWarning: Columns (11,26,101,111,142,143,217,218) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dpe-v2-logements-neufs.csv')


In [4]:
df.head()

,N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),...,Présence_production_PV_(0/1),Production_électricité_PV_kWh,Production_électricité_PV_kWh/m²,Surface_totale_capteurs_photovoltaïque,Nombre_module,Type_énergie_principale_chauffage,Type_énergie_principale_ECS,Déperditions_totales_logement,Déperditions_totales_bâtiment,Conso_5_usages_é_finale_énergie_n°1
0,2133N0419762I,2021-09-30,2021-09-29,2021-09-29,DPE NEUF logement : RT2012,NaN,2031-09-28,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2648.3
1,2262N1677399D,2022-07-22,2022-07-20,2022-07-20,DPE NEUF logement : RT2012,NaN,2032-07-19,2.1,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,NaN,NaN,1974.1
2,2333N0960129T,2023-03-22,2023-03-21,2023-03-21,DPE NEUF logement : RT2012,NaN,2033-03-20,2.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,NaN,NaN,1891.4
3,2474N0961053X,2024-03-16,2024-03-15,2024-03-14,DPE NEUF logement : RT2012,NaN,2034-03-14,2.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,NaN,NaN,2887.5
4,2317N4122363N,2023-12-01,2023-11-11,2023-11-11,DPE NEUF logement : RT2012,2317N4121938E,2033-11-10,2.3,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,Électricité,NaN,NaN,NaN,1300.3


In [5]:
for colonne in df.columns:
    print(colonne)

N°DPE
Date_réception_DPE
Date_établissement_DPE
Date_visite_diagnostiqueur
Modèle_DPE
N°_DPE_remplacé
Date_fin_validité_DPE
Version_DPE
N°_DPE_immeuble_associé
Appartement_non_visité_(0/1)
Méthode_application_DPE
N°_immatriculation_copropriété
N°_RPLS_logement
Invariant_fiscal_logement
Etiquette_DPE
Etiquette_GES
Type_bâtiment
Type_installation_chauffage
Type_installation_ECS_(général)
Hauteur_sous-plafond
Nombre_appartement
Nombre_niveau_immeuble
Nombre_niveau_logement
Surface_habitable_immeuble
Surface_tertiaire_immeuble
Surface_habitable_logement
Classe_inertie_bâtiment
Typologie_logement
Position_logement_dans_immeuble
Classe_altitude
Zone_climatique_
Adresse_brute
Nom__commune_(BAN)
Code_INSEE_(BAN)
N°_voie_(BAN)
Identifiant__BAN
Adresse_(BAN)
Code_postal_(BAN)
Score_BAN
Nom__rue_(BAN)
Coordonnée_cartographique_X_(BAN)
Coordonnée_cartographique_Y_(BAN)
Code_postal_(brut)
N°_étage_appartement
Nom_résidence
Complément_d'adresse_bâtiment
Cage_d'escalier
Complément_d'adresse_logement


In [6]:
df.shape

(783116, 220)

In [7]:
df=df.drop_duplicates()

In [143]:
new_df = df[[ 'Date_réception_DPE', 'Modèle_DPE','N°_DPE_remplacé','Date_fin_validité_DPE' ,'Version_DPE', 'N°_DPE_immeuble_associé', 'Invariant_fiscal_logement','Etiquette_DPE','Etiquette_GES','Type_bâtiment' ]]

In [144]:
new_df.dtypes

Date_réception_DPE            object
Modèle_DPE                    object
N°_DPE_remplacé               object
Date_fin_validité_DPE         object
Version_DPE                  float64
N°_DPE_immeuble_associé       object
Invariant_fiscal_logement     object
Etiquette_DPE                 object
Etiquette_GES                 object
Type_bâtiment                 object
dtype: object

In [145]:
new_df.shape


(783116, 10)

In [146]:
new_df.isnull().sum()


Date_réception_DPE                0
Modèle_DPE                        0
N°_DPE_remplacé              753245
Date_fin_validité_DPE             0
Version_DPE                       0
N°_DPE_immeuble_associé      579949
Invariant_fiscal_logement    760369
Etiquette_DPE                     0
Etiquette_GES                     0
Type_bâtiment                     0
dtype: int64

In [147]:
# Remplacer les valeurs non numériques par -1
new_df['N°_DPE_remplacé'] = pd.to_numeric(new_df['N°_DPE_remplacé'], errors='coerce').fillna(-1)
new_df['N°_DPE_immeuble_associé'] = pd.to_numeric(new_df['N°_DPE_immeuble_associé'], errors='coerce').fillna(-1)
new_df['Invariant_fiscal_logement'] = pd.to_numeric(new_df['Invariant_fiscal_logement'], errors='coerce').fillna(-1)
# Convertir les types de données
new_df['N°_DPE_remplacé'] = new_df['N°_DPE_remplacé'].astype(float)
new_df['N°_DPE_immeuble_associé'] = new_df['N°_DPE_immeuble_associé'].astype(float)
new_df['Invariant_fiscal_logement'] = new_df['Invariant_fiscal_logement'].astype(float)



C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\4132104531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['N°_DPE_remplacé'] = pd.to_numeric(new_df['N°_DPE_remplacé'], errors='coerce').fillna(-1)
C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\4132104531.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['N°_DPE_immeuble_associé'] = pd.to_numeric(new_df['N°_DPE_immeuble_associé'], errors='coerce').fillna(-1)
C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\41

In [148]:
new_df.isnull().sum()

Date_réception_DPE           0
Modèle_DPE                   0
N°_DPE_remplacé              0
Date_fin_validité_DPE        0
Version_DPE                  0
N°_DPE_immeuble_associé      0
Invariant_fiscal_logement    0
Etiquette_DPE                0
Etiquette_GES                0
Type_bâtiment                0
dtype: int64

In [149]:
new_df.shape

(783116, 10)

In [150]:
new_df.dtypes

Date_réception_DPE            object
Modèle_DPE                    object
N°_DPE_remplacé              float64
Date_fin_validité_DPE         object
Version_DPE                  float64
N°_DPE_immeuble_associé      float64
Invariant_fiscal_logement    float64
Etiquette_DPE                 object
Etiquette_GES                 object
Type_bâtiment                 object
dtype: object

In [152]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Conversion des colonnes de dates en types de données datetime
new_df['Date_réception_DPE'] = pd.to_datetime(new_df['Date_réception_DPE'])
new_df['Date_fin_validité_DPE'] = pd.to_datetime(new_df['Date_fin_validité_DPE'])

# Conversion des dates en nombres de jours écoulés depuis une date de référence
new_df['Date_réception_DPE'] = (new_df['Date_réception_DPE'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
new_df['Date_fin_validité_DPE'] = (new_df['Date_fin_validité_DPE'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')

# Sélection des colonnes catégorielles pour l'encodage one-hot
categorical_columns = ["Modèle_DPE", "Type_bâtiment"]

# Création d'un encodeur one-hot pour les colonnes catégorielles
encoder = OneHotEncoder()

# Création d'un ColumnTransformer pour appliquer l'encodage one-hot aux colonnes catégorielles
column_transformer = ColumnTransformer(
    transformers=[('onehot', encoder, categorical_columns)],
    remainder='passthrough'
)



C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\2746636827.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date_réception_DPE'] = pd.to_datetime(new_df['Date_réception_DPE'])
C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\2746636827.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date_fin_validité_DPE'] = pd.to_datetime(new_df['Date_fin_validité_DPE'])
C:\Users\DELL INSPIROON\AppData\Local\Temp\ipykernel_2052\2746636827.py:12: SettingWithCopyWarning: 
A value is 

In [153]:
new_df.dtypes

Date_réception_DPE             int64
Modèle_DPE                    object
N°_DPE_remplacé              float64
Date_fin_validité_DPE          int64
Version_DPE                  float64
N°_DPE_immeuble_associé      float64
Invariant_fiscal_logement    float64
Etiquette_DPE                 object
Etiquette_GES                 object
Type_bâtiment                 object
dtype: object

In [154]:

# Application du ColumnTransformer aux fonctionnalités pour obtenir les fonctionnalités transformées
X = new_df.drop(columns=["Etiquette_DPE", "Etiquette_GES"])  # Sélectionner les fonctionnalités sans les étiquettes
X_encoded = column_transformer.fit_transform(X)

# Séparation des données en ensembles d'entraînement et de test
y = new_df["Etiquette_GES"]  # Sélectionner la colonne des étiquettes
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Création et entraînement du modèle DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Évaluation du modèle
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.7126494023904383


In [165]:
# Sélectionner un exemple du dataset pour la prédiction (par exemple le premier exemple)
example = X_encoded[0]  # Premier exemple

# Prédire la classe de l'exemple
predicted_class = model.predict([example])[0]  # Prédire la classe de l'exemple

# Obtenir la vraie classe de l'exemple
true_class = df["Etiquette_GES"].iloc[0]  # Obtenir la vraie classe de l'exemple

# Afficher la classe prédite
print("Classe prédite:", predicted_class)

# Afficher la vraie classe
print("Vraie classe:", true_class)

# Comparer la prédiction avec la vraie classe
if predicted_class == true_class:
    print("Le modèle a correctement prédit la classe de l'exemple.")
else:
    print("Le modèle n'a pas correctement prédit la classe de l'exemple.")


Classe prédite: C
Vraie classe: C
Le modèle a correctement prédit la classe de l'exemple.


In [ ]:
import mlflow.sklearn

# Définir le nom de l'expérience MLflow
mlflow.set_experiment("Nom_de_votre_expérience")

# Enregistrer le modèle dans MLflow
with mlflow.start_run():
    # Enregistrer les paramètres du modèle
    mlflow.log_param("paramètre_1", valeur_paramètre_1)
    mlflow.log_param("paramètre_2", valeur_paramètre_2)
    # Enregistrer les métriques de performance
    mlflow.log_metric("métrique_1", valeur_métrique_1)
    mlflow.log_metric("métrique_2", valeur_métrique_2)
    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "nom_du_modèle")

In [167]:

experiment_name = "dpe_train_mlops"

# Adresse du serveur MLflow
remote_server_uri = "http://13.82.178.239:5001/"

# Configuration de l'adresse du serveur MLflow
mlflow.set_tracking_uri(remote_server_uri)

# Définir l'expérience MLflow
mlflow.set_experiment(experiment_name)

with mlflow.start_run():
    # Enregistrer les paramètres du modèle
    mlflow.log_param("random_state", 42)
    
    # Enregistrer les métriques de performance
    mlflow.log_metric("accuracy", accuracy)
    
    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "decision_tree_model")

2024/05/12 21:36:47 INFO mlflow.tracking.fluent: Experiment with name 'dpe_train_mlops' does not exist. Creating a new experiment.
c:\Users\DELL INSPIROON\AppData\Local\Programs\Python\Python39\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
